In [1]:
import os
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from collection import s3_download_file
from preprocessing import preprocess
from prediction import predict

torch.manual_seed(0)

In [2]:
S3_DATA_FILE = os.getenv("S3_DATA_FILE", "kagglecatsanddogs_5340.zip")
s3_download_file(S3_DATA_FILE, ".cache/data.zip")

In [7]:
!unzip -n -q .cache/data.zip -d .cache

In [6]:
# Directory path of your dataset
data_dir = '.cache/PetImages'

# Preprocess data: clean, resize,
# split into test and validation subsets...
train_loader, val_loader, dataset = preprocess(data_dir)

# Verify the class labels of the dataset
print("Classes:", dataset.classes)

# Verify sizes
for images, labels in train_loader:
    print("Features first batch size:", images.size())
    print("Labels first batch size:", labels.size())
    break

Deleted 0 corrupted images.
Classes: ['Cat', 'Dog']
Features first batch size: torch.Size([32, 3, 50, 50])
Labels first batch size: torch.Size([32])


In each iteration, the data loader returns a tuple of two batches (images and labels).
The images batch contains 32 images.
The labels batch contains the corresponding 32 labels for those images.

Each image has 50x50 pixels, with 3 color channels (RGB).

In [8]:
# Define the model (use a pre-trained ResNet and modify the final layer)
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
# Modify final layer to define 2 output classes: Cat and Dog
model.fc = nn.Linear(num_features, len(dataset.classes))

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using {device} for training")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using cuda:0 for training


In [9]:
# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}: \n Training Loss: {epoch_loss:.4f}')

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct.double() / len(val_loader.dataset)
    print(f' Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

Epoch 1/3, 
 Training Loss: 0.4265
 Validation Loss: 0.3337, Accuracy: 0.8445
Epoch 2/3, 
 Training Loss: 0.2884
 Validation Loss: 0.3252, Accuracy: 0.8413
Epoch 3/3, 
 Training Loss: 0.2316
 Validation Loss: 0.3019, Accuracy: 0.8795


In [11]:
# Smoke test the model
result = predict("test_cat.jpg", model)
print(result)

Cat
None


## Export to ONNX

In [12]:
# Generate a random Torch to specify
# the inputs dimensions expected by the model
first_batch_example = torch.randn(1, 3, 50, 50).to(device)
# Export to ONNX
torch.onnx.export(model, first_batch_example, "model.onnx")